## Eastern Demand Gathering Demo, V5
Authors: Bainan, Vicky, Dan M.

Last updated: 11/14/19

In [1]:
import pandas as pd
import numpy as np
import requests as r
import json
import pickle
import matplotlib as plt
from collections import Counter, defaultdict

from prereise.gather.demanddata.eia.clean_data import fix_dataframe_outliers, replace_with_shifted_demand
from prereise.gather.demanddata.eia.get_eia_data import get_ba_demand
from prereise.gather.demanddata.eia.map_ba import transform_ba_to_region, map_to_loadzone, map_grid_buses_to_county

from powersimdata.input.grid import Grid
import json

BAs are associated with abbreviations. Demand data from EIA is imported using an API Key.
Note that data for some BAs will not be found (i.e., YAD) and this will be reported by the download_map_demand.get_BA_demand function.

In [2]:
eastern_ba = ['SWPP','MISO','AECI','SPA','EEI','LGEE','OVEC','PJM','NYIS','ISNE','TVA','CPLW','DUK','YAD','CPLE','AEC','SOCO','SEPA','SC','TAL','SCEG','GVL','JEA','SEC','FPC','NSB','TEC','FMPP','FPL','HST']

In [3]:
start = pd.to_datetime('2016-01-01 00:00:00')
end = pd.to_datetime('2016-12-31 23:00:00')

eastern_demand = get_ba_demand(eastern_ba, start, end, 'd826fe7862968708eb6a3c2639d6da1b')
eastern_demand_2016 = eastern_demand.loc[start:end]
eastern_demand_2016

ERROR: EBA.EEI-ALL.D.H was found but has no data
ERROR: EBA.YAD-ALL.D.H not found. invalid series_id. For key registration, documentation, and examples see https://www.eia.gov/developer/
ERROR: EBA.SEPA-ALL.D.H not found. invalid series_id. For key registration, documentation, and examples see https://www.eia.gov/developer/


,SWPP,MISO,AECI,SPA,LGEE,OVEC,PJM,NYIS,ISNE,TVA,...,SCEG,GVL,JEA,SEC,FPC,NSB,TEC,FMPP,FPL,HST
2016-01-01 00:00:00+00:00,30875.0,78481.0,2917.0,68.0,4280.0,NaN,95072.0,19726.0,16063.0,18742.0,...,2637.0,270.0,NaN,175.0,7389.0,65.0,2969.0,2507.0,18036.0,84.0
2016-01-01 01:00:00+00:00,31457.0,77902.0,2946.0,83.0,4245.0,NaN,92298.0,19008.0,15273.0,18555.0,...,2558.0,258.0,NaN,164.0,6931.0,61.0,2778.0,2386.0,16990.0,79.0
2016-01-01 02:00:00+00:00,30969.0,75821.0,2910.0,58.0,4078.0,NaN,89416.0,18340.0,14568.0,18180.0,...,2487.0,244.0,NaN,152.0,0.0,53.0,2575.0,2226.0,15650.0,73.0
2016-01-01 03:00:00+00:00,30397.0,73863.0,2878.0,80.0,3935.0,NaN,86496.0,17557.0,13850.0,17744.0,...,2374.0,229.0,NaN,140.0,5830.0,51.0,2397.0,2049.0,14406.0,66.0
2016-01-01 04:00:00+00:00,29818.0,71919.0,2829.0,69.0,3793.0,NaN,83405.0,16772.0,13108.0,17306.0,...,2256.0,213.0,NaN,128.0,5405.0,48.0,2223.0,1908.0,13389.0,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-12-31 19:00:00+00:00,26723.0,69959.0,2178.0,70.0,4090.0,80.0,89370.0,17863.0,14716.0,18673.0,...,2727.0,183.0,1275.0,124.0,5087.0,NaN,2099.0,NaN,12686.0,53.0
2016-12-31 20:00:00+00:00,26109.0,68689.0,2094.0,58.0,4038.0,80.0,90063.0,17847.0,14714.0,18408.0,...,2676.0,182.0,1234.0,115.0,4998.0,NaN,2047.0,NaN,12760.0,54.0
2016-12-31 21:00:00+00:00,25618.0,68160.0,2035.0,60.0,3957.0,80.0,90826.0,18117.0,14965.0,18052.0,...,2651.0,182.0,1220.0,115.0,4949.0,NaN,2061.0,NaN,12858.0,56.0
2016-12-31 22:00:00+00:00,25441.0,68204.0,2035.0,64.0,4014.0,80.0,92672.0,19122.0,15996.0,18122.0,...,2714.0,185.0,1230.0,116.0,4981.0,NaN,2089.0,NaN,13248.0,57.0


Some BAs are very small. In these cases, BAs are agregated into BA areas as follows:

In [4]:
BA_area ={
    'ISNE':['ISNE'],
    'NYIS':['NYIS'],
    'PJM':['PJM','OVEC'],
    'AEC':['AEC'],
    'SOCO':['SOCO'],
    'TVA_LGEE':['TVA','LGEE'],
    'Florida':['TAL','JEA','GVL','SEC','FPC','NSB','FMPP','TEC','FPL','HST'],
    'MISO':['MISO','AECI'],
    'SWPP':['SWPP','SPA'],
    'Carolina':['CPLW','DUK','SCEG','SC','CPLE']
}

Use adjacent demand data to fill missing values using a series of rules:
1. Monday: look forward one day
2. Tues - Thurs: average of look forward one day and look back one day
3. Fri: look back one day
4. Sat: look forward one day
5. Sun: look back one day

If data is still missing after applying the above rules, week ahead and week behind data is used
1. Monday: look forward two days 
2. Tues: look forward two days
3. Wed: average of look forward two days and look back two days
4. Thurs: look back two days
5. Fri: look back two days
6. Sat - Sun: average of look back one week and look forward one week

If data is still missing after applying the above rules, week ahead and week behind data is used:
1. Mon - Sun: average of look back one week and look forward one week

In [5]:
filled_eastern_demand_2016 = replace_with_shifted_demand(eastern_demand_2016, start, end)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/Users/mondreicka/Intellectual_Ventures/PreREISE/prereise/gather/demanddata/eia/clean_data.py:183: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shifted_demand_ba, baName, day_map)
/Users/mondreicka/Intellectual_Ventures/PreREISE/prereise/gather/demanddata/eia/clean_data.py:185: SettingWithCopyWarning: 
A value is trying to be set on a copy

In [6]:
for baName in filled_eastern_demand_2016.columns:
    print(baName)
    print(filled_eastern_demand_2016.loc[filled_eastern_demand_2016[baName].isnull()])

SWPP
Empty DataFrame
Columns: [SWPP, MISO, AECI, SPA, LGEE, OVEC, PJM, NYIS, ISNE, TVA, CPLW, DUK, CPLE, AEC, SOCO, SC, TAL, SCEG, GVL, JEA, SEC, FPC, NSB, TEC, FMPP, FPL, HST]
Index: []

[0 rows x 27 columns]
MISO
Empty DataFrame
Columns: [SWPP, MISO, AECI, SPA, LGEE, OVEC, PJM, NYIS, ISNE, TVA, CPLW, DUK, CPLE, AEC, SOCO, SC, TAL, SCEG, GVL, JEA, SEC, FPC, NSB, TEC, FMPP, FPL, HST]
Index: []

[0 rows x 27 columns]
AECI
Empty DataFrame
Columns: [SWPP, MISO, AECI, SPA, LGEE, OVEC, PJM, NYIS, ISNE, TVA, CPLW, DUK, CPLE, AEC, SOCO, SC, TAL, SCEG, GVL, JEA, SEC, FPC, NSB, TEC, FMPP, FPL, HST]
Index: []

[0 rows x 27 columns]
SPA
Empty DataFrame
Columns: [SWPP, MISO, AECI, SPA, LGEE, OVEC, PJM, NYIS, ISNE, TVA, CPLW, DUK, CPLE, AEC, SOCO, SC, TAL, SCEG, GVL, JEA, SEC, FPC, NSB, TEC, FMPP, FPL, HST]
Index: []

[0 rows x 27 columns]
LGEE
Empty DataFrame
Columns: [SWPP, MISO, AECI, SPA, LGEE, OVEC, PJM, NYIS, ISNE, TVA, CPLW, DUK, CPLE, AEC, SOCO, SC, TAL, SCEG, GVL, JEA, SEC, FPC, NSB, TEC, 

Transform BA demand to generalized BA areas by summing appropriate columns

In [7]:
eastern_agg_demand = transform_ba_to_region(eastern_demand_2016, BA_area)
eastern_agg_demand

ISNE regional demand was summed from ['ISNE']

NYIS regional demand was summed from ['NYIS']

PJM regional demand was summed from ['OVEC', 'PJM']

AEC regional demand was summed from ['AEC']

SOCO regional demand was summed from ['SOCO']

TVA_LGEE regional demand was summed from ['LGEE', 'TVA']

Florida regional demand was summed from ['NSB', 'FPC', 'GVL', 'TEC', 'SEC', 'HST', 'JEA', 'FPL', 'TAL', 'FMPP']

MISO regional demand was summed from ['AECI', 'MISO']

SWPP regional demand was summed from ['SWPP', 'SPA']

Carolina regional demand was summed from ['CPLE', 'CPLW', 'DUK', 'SC', 'SCEG']



,ISNE,NYIS,PJM,AEC,SOCO,TVA_LGEE,Florida,MISO,SWPP,Carolina
2016-01-01 00:00:00+00:00,16063.0,19726.0,95072.0,492.0,23927.0,23022.0,31808.0,81398.0,30943.0,22878.0
2016-01-01 01:00:00+00:00,15273.0,19008.0,92298.0,488.0,24347.0,22800.0,29950.0,80848.0,31540.0,22097.0
2016-01-01 02:00:00+00:00,14568.0,18340.0,89416.0,465.0,23574.0,22258.0,21262.0,78731.0,31027.0,21357.0
2016-01-01 03:00:00+00:00,13850.0,17557.0,86496.0,444.0,22762.0,21679.0,25439.0,76741.0,30477.0,20466.0
2016-01-01 04:00:00+00:00,13108.0,16772.0,83405.0,429.0,21857.0,21099.0,23625.0,74748.0,29887.0,19469.0
...,...,...,...,...,...,...,...,...,...,...
2016-12-31 19:00:00+00:00,14716.0,17863.0,89450.0,545.0,26496.0,22763.0,21783.0,72137.0,26793.0,24925.0
2016-12-31 20:00:00+00:00,14714.0,17847.0,90143.0,528.0,26294.0,22446.0,21659.0,70783.0,26167.0,24653.0
2016-12-31 21:00:00+00:00,14965.0,18117.0,90906.0,513.0,26043.0,22009.0,21707.0,70195.0,25678.0,24634.0
2016-12-31 22:00:00+00:00,15996.0,19122.0,92752.0,521.0,26012.0,22136.0,22178.0,70239.0,25505.0,25303.0


In [15]:
eastern_agg_demand.to_pickle("./eastern_agg_demand.pkl")

Indentify and fix outliers. Outliers defined as demand hours for which the ramp rate (difference between two hours) is greater than 3 standard deviations of the ramp rates for the BA across all hours in the year. 
Demand data for hours identified as outliers are replaced with the average of the neighboring non-outlier hour's demand.
Stretches of >5 consecutive outlier hours are flagged.

In [8]:
eastern_fix_outliers = fix_dataframe_outliers(eastern_agg_demand)

1357 104873.0 104873.0
1358 137033.0 104409.0
1359 103945.0 103945.0
1734 67360.0 67360.0
1735 67.0 66320.0
1736 65280.0 65280.0
1756 76763.0 76763.0
1757 52.0 52.0
1780 49.0 49.0
1781 74584.0 74584.0
7468 70188.0 70188.0
7469 0.0 69438.5
7470 68689.0 68689.0
2410 19206.0 19206.0
2411 26343.0 19742.0
2412 20278.0 20278.0
701 23378.0 23378.0
702 4517.0 22913.0
703 22448.0 22448.0
704 22324.0 22324.0
705 3964.0 22365.0
706 22406.0 22406.0
1735 14850.0 14850.0
1736 2638.0 2638.0
1757 2897.0 2897.0
1758 15124.0 15124.0
7469 16362.0 16362.0
7470 4106.0 15976.0
7471 15590.0 15590.0
8237 25897.0 25897.0
8238 5649.0 25647.5
8239 25398.0 25398.0
8477 28120.0 28120.0
8478 7081.0 27567.5
8479 27015.0 27015.0
1564 18918.0 18918.0
1565 28308.0 18278.0
1566 17638.0 17638.0
1876 31575.0 31575.0
1877 21050.0 21050.0
3912 33901.0 33901.0
3913 16354.0 16354.0
3916 13220.0 13220.0
3917 24792.0 24792.0
4194 42510.0 42510.0
4195 21544.0 21544.0
4204 15518.0 15518.0
4205 29081.0 29081.0
4850 38735.0 38735.0

In [9]:
eastern_fix_outliers 

,ISNE,NYIS,PJM,AEC,SOCO,TVA_LGEE,Florida,MISO,SWPP,Carolina
2016-01-01 00:00:00+00:00,16063.0,19726.0,95072.0,492.0,23927.0,23022.0,31808.0,81398.0,30943.0,22878.0
2016-01-01 01:00:00+00:00,15273.0,19008.0,92298.0,488.0,24347.0,22800.0,29950.0,80848.0,31540.0,22097.0
2016-01-01 02:00:00+00:00,14568.0,18340.0,89416.0,465.0,23574.0,22258.0,21262.0,78731.0,31027.0,21357.0
2016-01-01 03:00:00+00:00,13850.0,17557.0,86496.0,444.0,22762.0,21679.0,25439.0,76741.0,30477.0,20466.0
2016-01-01 04:00:00+00:00,13108.0,16772.0,83405.0,429.0,21857.0,21099.0,23625.0,74748.0,29887.0,19469.0
...,...,...,...,...,...,...,...,...,...,...
2016-12-31 19:00:00+00:00,14716.0,17863.0,89450.0,545.0,26496.0,22763.0,21783.0,72137.0,26793.0,24925.0
2016-12-31 20:00:00+00:00,14714.0,17847.0,90143.0,528.0,26294.0,22446.0,21659.0,70783.0,26167.0,24653.0
2016-12-31 21:00:00+00:00,14965.0,18117.0,90906.0,513.0,26043.0,22009.0,21707.0,70195.0,25678.0,24634.0
2016-12-31 22:00:00+00:00,15996.0,19122.0,92752.0,521.0,26012.0,22136.0,22178.0,70239.0,25505.0,25303.0


## Get bus to BA area mapping via county
Author: Bainan  
Last updated: 11/14/19

Warning: the following steps take around 3 hours to finish. The results are stored in bus_ba_map.csv for future use, as implemented below:

First, get Eastern grid. 

In [18]:
eastern = Grid(['Eastern'])

--> Loading Eastern interconnect
Loading zone
Loading sub
Loading bus2sub
Loading bus
Loading plant
Loading plant cost
Loading branch
Loading DC line


In [15]:
#bus_ba_map, bus_no_BA_match = map_grid_buses_to_county(eastern)*

#bus_ba_map.to_csv('bus_ba_map.csv')*

The csv file is created at this stage to aviod time-intensive reruns.

In [20]:
bus_ba_map = pd.read_csv('bus_ba_map.csv',index_col=0)

The following step loads a file that stores list of counties in each BA area territory. The file is created mannually based on external references.

In [22]:
data = json.load(open('EasternBAtoCountyDraft6.txt'))
ba_county_list = {}
for val in data['groups'].values():
    ba_county_list[val['label']] = set(val['paths'])

In [ ]:
for index,row in bus_ba_map.iterrows():
    for BA,clist in ba_county_list.items():
        try:
            county = row['County'].replace(' ','_')
            county = county.replace('.','')
            county = county.replace('-','')
            county = county.replace('\'','_')
            if row['County'] == 'LaSalle__IL':
                county = 'La_Salle__IL'
            if row['County'] == 'Lac Qui Parle__MN':
                county = 'Lac_qui_Parle__MN'
            if row['County'] == 'Baltimore__MD':
                county = 'Baltimore_County__MD'
            if row['County'] == 'District of Columbia__DC':
                county = 'Washington__DC'
            if row['County'] == 'St. Louis City__MO':
                county = 'St_Louis_Co__MO'
            if county in clist:
                bus_ba_map.loc[index,'BA'] = BA
                break
        except:
            continue

In [ ]:
bus_ba_map

Check if there are buses where BA is empty/not found, which is due to bus being outside of United States. These were fixed manually by assigning to the nearest county. This procedure is commented out as it was used just one time. 

Check if there are any buses where BA is null, which is due to a name mis-match.

Then check if there are buses where county is null. 

In [23]:
#bus_no_BA_match = list(bus_ba_map[~bus_ba_map['BA'].astype(bool)].index)

bus_no_BA_match = list(bus_ba_map[bus_ba_map['BA'].isna()].index)
bus_no_BA_match

[]

In [ ]:
bus_no_county_match = list(bus_ba_map[bus_ba_map['County'].isna()].index)
bus_no_county_match

Add zone name into the data frame for reference.

In [24]:
bus_ba_map.loc[:,'zone_name'] = eastern.bus[eastern.bus['Pd']>0]['zone_id'].apply(lambda x: eastern.id2zone[x])

Fix mismatch county names in Virginia and Maryland

In [25]:
for ind in bus_no_BA_match:
    if bus_ba_map.loc[ind,'zone_name'] in {'Virginia Mountains','West Virginia','Virginia Tidewater','Maryland'}:
        bus_ba_map.loc[ind,'BA'] = 'PJM'

## Manually assign outliers (outside US territory) to the nearest BA
Bus no county match: [91: ISNE, 7991: NYIS, 7992: NYIS, 8707: NYIS, 8708: NYIS, 40644: MISO]  
Assign the rest no-ba-match buses to SWPP.  
Assign buses in ERCOT Texas to SWPP or MISO based on the location by observation.  
Make BA code cosnistent with EIA data source.  
Overwrite the final results with two new columns for each row (bus): BA and zone_name. 

In [26]:
# Manually assign outliers (outside US territory) to the nearest BA
# bus no county match: [91: ISNE, 7991: NYIS, 7992: NYIS, 8707: NYIS, 8708: NYIS, 40644: MISO]
bus_ba_map.loc[91,'BA'] = 'ISNE'
bus_ba_map.loc[7991,'BA'] = 'NYIS'
bus_ba_map.loc[7992,'BA'] = 'NYIS'
bus_ba_map.loc[8707,'BA'] = 'NYIS'
bus_ba_map.loc[8708,'BA'] = 'NYIS'
bus_ba_map.loc[40644,'BA'] = 'MISO'

# Assign the rest no-ba-match buses to SWPP
for ind in bus_no_BA_match:
    bus_ba_map.loc[ind,'BA'] = 'SWPP'

# Assign buses in ERCOT Texas to SWPP or MISO based on the location by observation
miso_tx_ind = bus_ba_map[(bus_ba_map['BA']=='ERCOT Texas') & (bus_ba_map['zone_name']=='East Texas') & ((bus_ba_map['County']=='Montgomery__TX') | (bus_ba_map['County']=='Walker__TX'))].index
for ind in bus_ba_map[bus_ba_map['BA']=='ERCOT Texas'].index:
    if ind in miso_tx_ind:
        bus_ba_map.loc[ind,'BA']='MISO'
    else:
        bus_ba_map.loc[ind,'BA']='SWPP'

# Make BA Code consistent with EIA data source
ba_code_fix = {'ISONE':'ISNE','NYISO':'NYIS'}
bus_ba_map.replace(ba_code_fix,inplace=True)

# The csv file is overwritten by the final results with two new columns for each row (bus): BA and zone_name.
bus_ba_map.to_csv('bus_ba_map.csv')

## Start here if "bus_ba_map.csv" already exists

In [10]:
bus_map = pd.read_csv('bus_ba_map.csv')
bus_map.set_index('bus_id')

,Pd,lat,lon,County,BA,zone_name
bus_id,,,,,,
3,9.97,43.976116,-70.221052,Androscoggin__ME,ISNE,Maine
9,11.48,44.100992,-70.305323,Androscoggin__ME,ISNE,Maine
10,9.97,44.179913,-70.185998,Androscoggin__ME,ISNE,Maine
11,9.97,44.143322,-70.224057,Androscoggin__ME,ISNE,Maine
12,3.06,44.020747,-70.180492,Androscoggin__ME,ISNE,Maine
...,...,...,...,...,...,...
69946,1.70,48.307278,-106.272727,Valley__MT,MISO,Montana Eastern
69948,0.85,48.222633,-106.316038,Valley__MT,MISO,Montana Eastern
69949,0.66,48.932800,-106.642307,Valley__MT,MISO,Montana Eastern


Transform to load zone using the demand that has missing data filled and outliers fixed, prepared from the first steps in the demo notebook.

In [11]:
final_eastern_demand =  map_to_loadzone(eastern_fix_outliers, bus_map)
final_eastern_demand

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



,Alabama,Florida Panhandle,Georgia North,Georgia South,North Carolina,South Carolina,Tennessee,Virginia Mountains,Western North Carolina,Florida North,...,Pennsylvania Eastern,Pennsylvania Western,Delaware,Maryland,Virginia Tidewater,West Virginia,Kansas,New Mexico Eastern,Oklahoma,Texas Panhandle
2016-01-01 00:00:00+00:00,8786.926545,2157.601335,12360.071456,5113.259326,11944.598509,8728.645513,13375.114737,3882.419152,2573.601414,10833.668568,...,11126.541138,5560.185292,1457.761877,9773.362600,9707.254327,3013.974465,6110.042669,316.149204,8405.379423,1168.394506
2016-01-01 01:00:00+00:00,8858.602118,2139.145162,12552.398497,5199.036064,11538.979460,8430.670508,13245.715847,3770.263478,2488.158045,10200.841726,...,10801.826942,5397.876583,1415.227467,9488.196538,9424.017165,2926.033061,6227.927020,322.248841,8567.548945,1190.936972
2016-01-01 02:00:00+00:00,8590.256665,1891.201541,12159.946414,5029.572011,11153.502582,8148.338238,12930.169840,3652.948377,2405.781470,7241.746136,...,10464.506373,5229.289357,1371.037067,9191.928120,9129.752745,2834.667838,6126.629412,317.007444,8428.197245,1171.566310
2016-01-01 03:00:00+00:00,8310.449246,1940.190500,11747.044744,4858.720649,10691.865112,7808.394924,12593.612601,3533.721533,2306.922149,8664.414446,...,10122.689492,5058.423900,1326.264003,8891.753318,8831.608363,2742.097939,6018.025739,311.388013,8278.794838,1150.798545
2016-01-01 04:00:00+00:00,8014.223867,1844.411223,11289.062901,4664.686370,10176.108641,7428.009419,12256.314016,3407.506096,2196.595114,8046.573815,...,9760.875341,4877.575143,1278.868956,8573.999786,8516.004157,2644.106995,5901.523616,305.359896,8118.526802,1128.520396
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-12-31 19:00:00+00:00,9423.935874,2076.668995,13593.114373,5653.858294,12949.189476,9509.637617,13205.814867,3662.768437,2793.661113,7419.196505,...,10468.263763,5231.025467,1371.558397,9195.423306,9133.224289,2835.745707,5290.578588,273.748041,7278.070352,1011.692273
2016-12-31 20:00:00+00:00,9323.285873,2061.683098,13484.372524,5610.595494,12815.456718,9405.861431,13022.592680,3688.760813,2762.922003,7376.962636,...,10549.294293,5271.471540,1382.184333,9266.663422,9203.982527,2857.715207,5166.967861,267.352106,7108.023248,988.054780
2016-12-31 21:00:00+00:00,9199.594474,2047.825022,13348.687256,5557.660664,12809.359528,9398.612359,12772.201353,3717.662416,2759.024618,7393.311230,...,10638.641470,5316.153012,1393.883596,9345.099509,9281.888063,2881.903848,5070.409322,262.355921,6975.190926,969.590348
2016-12-31 22:00:00+00:00,9214.268403,2057.031563,13339.979956,5552.875959,13154.403204,9653.855993,12846.132097,3792.527215,2831.933288,7553.731813,...,10854.968898,5424.438616,1422.188758,9534.867552,9470.372490,2940.425778,5036.248530,260.588354,6928.197078,963.057941


Load in Eastern Grid and transform column names into corresponding loadzone ids

In [12]:
eastern = Grid(['Eastern'])
final_eastern_demand.columns = [eastern.zone2id[col] for col in final_eastern_demand.columns]
final_eastern_demand = final_eastern_demand.sort_index(axis = 1)
final_eastern_demand

--> Loading Eastern interconnect
Loading zone
Loading sub
Loading bus2sub
Loading bus
Loading plant
Loading plant cost
Loading branch
Loading DC line


,1,2,3,4,5,6,7,8,9,10,...,43,44,45,46,47,48,49,50,51,52
2016-01-01 00:00:00+00:00,1342.820998,1440.655575,600.763115,7581.562212,910.213388,4209.789191,13682.014219,6569.107335,11713.791325,11126.541138,...,10853.450041,2371.772581,1168.394506,316.149204,8405.379423,6110.042669,4447.529440,1373.158280,1366.952944,149.880227
2016-01-01 01:00:00+00:00,1276.779250,1369.802192,571.235771,7208.690759,865.447866,4003.605235,13191.418166,6330.139172,11367.967452,10801.826942,...,10798.385096,2382.624748,1190.936972,322.248841,8567.548945,6227.927020,4529.357442,1377.295717,1374.289820,149.433565
2016-01-01 02:00:00+00:00,1217.843260,1306.572273,544.883366,6875.938387,825.498888,3819.512768,12731.570544,6107.744102,11010.890784,10464.506373,...,10522.721761,2330.668354,1171.566310,317.007444,8428.197245,6126.629412,4454.181626,1346.438528,1344.736901,145.740370
2016-01-01 03:00:00+00:00,1157.820508,1242.176413,518.037295,6537.050155,784.813262,3631.675821,12198.174558,5847.184973,10646.093831,10122.689492,...,10262.059399,2279.568946,1150.798545,311.388013,8278.794838,6018.025739,4374.108813,1316.304481,1315.563397,142.221137
2016-01-01 04:00:00+00:00,1095.791423,1175.628045,490.295684,6186.834183,742.767670,3437.642565,11661.456256,5585.918592,10261.177675,9760.875341,...,10000.116213,2227.086383,1128.520396,305.359896,8118.526802,5901.523616,4288.478577,1285.473076,1285.540547,138.669091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-12-31 19:00:00+00:00,1230.215638,1319.846072,550.369002,6945.792786,833.885340,3856.056040,12429.282223,5949.516634,11001.349786,10468.263763,...,9604.338286,2080.936274,1011.692273,273.748041,7278.070352,5290.578588,3854.146105,1206.451556,1198.486470,132.385658
2016-12-31 20:00:00+00:00,1230.048444,1319.666696,550.293181,6944.848807,833.772010,3855.485732,12427.158302,5944.373335,11082.201766,10549.294293,...,9421.276151,2037.772776,988.054780,267.352106,7108.023248,5166.967861,3764.718808,1181.757877,1173.460099,129.814354
2016-12-31 21:00:00+00:00,1251.031328,1342.178341,559.677808,7063.318092,847.994979,3921.137495,12608.264844,6034.163572,11179.378799,10638.641470,...,9336.855999,2011.787603,969.590348,262.355921,6975.190926,5070.409322,3695.744353,1167.420144,1158.127294,128.545226
2016-12-31 22:00:00+00:00,1337.219989,1434.646491,598.217981,7549.938937,906.416818,4190.452183,13271.103871,6368.157596,11424.466326,10854.968898,...,9338.422445,2006.743406,963.057941,260.588354,6928.197078,5036.248530,3671.815456,1165.004751,1154.965366,128.493010
